In [1]:
# Set country to ADB country code of country
country = 'RUS'

In [2]:
################################################################################################
####                               Import libraries and functions                           ####
################################################################################################
import wbdata as wb                                       
import pandas as pd
import numpy as np                                     
import datetime as dt                                     
import docx                                               
from docx.shared import Cm       
from docx.shared import Pt                         
from docx.enum.text import WD_ALIGN_PARAGRAPH             
from matplotlib import pyplot as plt 
from matplotlib import ticker
import random
# import Haver
import calendar
import re
# from pandasgui import show

In [3]:
################################################################################################
####                   Set WB variables to get + Years + Country names                      ####
################################################################################################

# Create a dictionary of all indicators to be scraped from the World Bank API
all_indicators = {'NY.GDP.MKTP.KD.ZG': 'GDP growth (%)',
                  'NE.CON.PRVT.ZS': 'Private consumption (% of GDP)', 
                  'NE.CON.GOVT.ZS': 'Government expenditure (% of GDP)',
                  'NE.GDI.TOTL.ZS': 'Gross capital formation (% of GDP)', 
                  'NE.EXP.GNFS.ZS': 'Exports (% of GDP)',
                  'NE.IMP.GNFS.ZS': 'Imports (% of GDP)',
                  'NE.CON.PRVT.KD.ZG': 'Private consumption (annual growth, %)', 
                  'NE.CON.GOVT.KD.ZG': 'Government expenditure (annual growth, %)',
                  'NE.GDI.TOTL.KD.ZG': 'Gross capital formation (annual growth, %)',
                  'NE.EXP.GNFS.KD.ZG': 'Exports (annual growth, %)',
                  'NE.IMP.GNFS.KD.ZG': 'Imports (annual growth, %)',
                  'NE.EXP.GNFS.CD': 'Exports (current prices, USD)',
                  'NE.IMP.GNFS.CD': 'Imports (current prices, USD)',
                  'NV.AGR.TOTL.ZS': 'Agriculture (% of GDP)',
                  'NV.AGR.TOTL.KD.ZG': 'Agriculture (annual growth, %)',
                  'NV.IND.TOTL.ZS': 'Industry (including construction) (% of GDP)',
                  'NV.IND.TOTL.KD.ZG': 'Industry (including construction) (annual growth, %)',
                  'NV.SRV.TOTL.ZS': 'Services (% of GDP)',
                  'NV.SRV.TOTL.KD.ZG': 'Services (annual growth, %)',
                  'SL.UEM.TOTL.NE.ZS': 'Unemployment rate (%)',
                  'FP.CPI.TOTL.ZG': 'Inflation rate (%)',
                  'BN.CAB.XOKA.GD.ZS': 'Current account balance (% of GDP)',
                  'BN.GSR.GNFS.CD': 'Net trade in goods and services (current USD)',
                  'BN.GSR.MRCH.CD': 'Net trade in goods (current USD)',
                  'FI.RES.TOTL.CD': 'Total reserves (includes gold, current USD)',
                  'FI.RES.TOTL.MO': 'Total reserves in months of imports'}

# Set the time period
data_date = dt.datetime(dt.date.today().year - 7, 1, 1), dt.datetime(dt.date.today().year - 2, 1, 1)

# Read Excel file of country codes and country names
country_codes = pd.read_excel('all-data.xlsx', sheet_name='data', index_col='adb_code')

# Read Excel file of Haver data
quarterly_data = pd.read_excel('quarterly_monthly_data.xlsx', sheet_name=country, header=8, index_col=0, usecols='J:Q')
quarterly_data.index = pd.PeriodIndex(quarterly_data.index, freq='Q').to_timestamp()
monthly_data = pd.read_excel('quarterly_monthly_data.xlsx', sheet_name=country, header=8, index_col=0, usecols='Y:AC', parse_dates=['month_year'])
policy_rate_annual = pd.read_excel('quarterly_monthly_data.xlsx', sheet_name=country, header=8, index_col=0, usecols='AH:AI', parse_dates=['year']).dropna()

In [4]:
################################################################################################
####                              Scrape + Prepare WB Yearly Data                           ####
################################################################################################

# Scrape data from the World Bank API (country in [] because want to access the 'RUS' row in country_codes DF, alternative to iat, etc.)
df_all = wb.get_dataframe(indicators=all_indicators, country=country_codes.iso_code[country], data_date=data_date, convert_date=True, source=2, cache=False)

# Sort data by year # Syntax: {} = dictionary; [] = list; () = arguments for functions
df_all = df_all.sort_index() 

# Generate share of net exports in GDP
net_exports_share = df_all['Exports (% of GDP)'] - df_all['Imports (% of GDP)']

# Insert 'net_exports_share' as the 6th column, labeled 'Net exports (% of GDP)'
df_all.insert(6, 'Net exports (% of GDP)', net_exports_share)

# Generate net exports annual growth
net_exports_growth = (df_all['Exports (current prices, USD)'] - df_all['Imports (current prices, USD)']).pct_change() * 100

# Insert 'net_exports_growth' as the 12th column, labeled 'Net exports (annual growth, %)'
df_all.insert(12, 'Net exports (annual growth, %)', net_exports_growth)

# Generate contributions to GDP growth using a loop
components = ['Private consumption', 'Government expenditure', 'Gross capital formation', 
              'Exports', 'Imports', 'Agriculture', 'Industry (including construction)', 'Services']
for item in components:
    df_all[f'{item} (contribution, pp)'] = df_all[f'{item} (% of GDP)'].shift(1) / 100 * df_all[f'{item} (annual growth, %)']

# Generate net export contribution to GDP growth
df_all['Net exports (contribution, pp)'] = df_all['Exports (contribution, pp)'] - df_all['Imports (contribution, pp)']

# Round to one decimal
df_all = df_all.round(1)

# Create dataframe for top three sources of growth on the demand/production side
demand_components = ['Private consumption', 'Government expenditure', 'Gross capital formation', 
                     'Net exports']
production_components = ['Agriculture', 'Industry (including construction)', 'Services']
components = demand_components + production_components

#  test: df_all['GDP growth (%)'].last('Y').values[0] = -1

# Create dataframe for top sources of growth on the demand side
top_demand_contributions = []
if df_all['GDP growth (%)'].last('Y').values[0] > 0:
    top_demand_contributions = df_all.last('Y')[[f'{item} (contribution, pp)' for item in demand_components]].transpose().sort_values(by=df_all.last('Y').index[0], ascending=False)
else:
    top_demand_contributions = df_all.last('Y')[[f'{item} (contribution, pp)' for item in demand_components]].transpose().sort_values(by=df_all.last('Y').index[0])
top_demand_contributions.index = [index.replace(' (contribution, pp)', '') for index in top_demand_contributions.index]

# Create dataframe for top shares of GDP on the demand side
top_demand_shares = df_all.last('Y')[[f'{item} (% of GDP)' for item in demand_components]].transpose()
top_demand_shares.index = [index.replace(' (% of GDP)', '') for index in top_demand_shares.index]
top_demand_shares = top_demand_shares.reindex(top_demand_contributions.index)

# Create dataframe for top growth rates of GDP components on the demand side
top_demand_growth = []
if df_all['GDP growth (%)'].last('Y').values[0] > 0:
    top_demand_growth = df_all.last('Y')[[f'{item} (annual growth, %)' for item in demand_components]].transpose().sort_values(by=df_all.last('Y').index[0], ascending=False)
else:
    top_demand_growth = df_all.last('Y')[[f'{item} (annual growth, %)' for item in demand_components]].transpose().sort_values(by=df_all.last('Y').index[0])
top_demand_growth.index = [index.replace(' (annual growth, %)', '') for index in top_demand_growth.index]

# Create dataframe for top sources of growth on the production side
top_production_contributions = []
if df_all['GDP growth (%)'].last('Y').values[0] > 0:
    top_production_contributions = df_all.last('Y')[[f'{item} (contribution, pp)' for item in production_components]].transpose().sort_values(by=df_all.last('Y').index[0], ascending=False)
else:
    top_production_contributions = df_all.last('Y')[[f'{item} (contribution, pp)' for item in production_components]].transpose().sort_values(by=df_all.last('Y').index[0])
top_production_contributions.index = [index.replace(' (contribution, pp)', '') for index in top_production_contributions.index]

# Create dataframe for top shares of GDP on the supply side
top_production_shares = df_all.last('Y')[[f'{item} (% of GDP)' for item in production_components]].transpose()
top_production_shares.index = [index.replace(' (% of GDP)', '') for index in top_production_shares.index]
top_production_shares = top_production_shares.reindex(top_production_contributions.index)

# Create dataframe for top growth rates of GDP components on the production side
top_production_growth = []
if df_all['GDP growth (%)'].last('Y').values[0] > 0:
    top_production_growth = df_all.last('Y')[[f'{item} (annual growth, %)' for item in production_components]].transpose().sort_values(by=df_all.last('Y').index[0], ascending=False)
else:
    top_production_growth = df_all.last('Y')[[f'{item} (annual growth, %)' for item in production_components]].transpose().sort_values(by=df_all.last('Y').index[0])
top_production_growth.index = [index.replace(' (annual growth, %)', '') for index in top_production_growth.index]

In [5]:
################################################################################################
####                                Create & Title Word Doc                                 ####
################################################################################################

# Create .docx
doc = docx.Document()

# Add title (heading level 0)
doc.add_heading(f'{country_codes.country_name[country]}: Recent Economic Developments and Outlook', level=0)

In [6]:
################################################################################################
####                                    GDP Growth Section                                  ####
################################################################################################

# 1 - Set placeholders + Qualify changes
# wout/ placeholders, Python would try and compare objects (and not values) and it would fail
change = ''
if df_all['GDP growth (%)'].last('Y').values[0] > 0:
    change = random.choice(['increased', 'picked up', 'rose'])
elif df_all['GDP growth (%)'].last('Y').values[0] == 0:
    change = random.choice(['remained at', 'stayed at'])
else:
    change = random.choice(['decreased', 'contracted', 'slowed down'])

# 2 - Heading qualifying GDP growth
doc.add_heading(f'GDP growth in {dt.date.today().year - 2} {change}', level=1)

# 3 - State the change
p = doc.add_paragraph(f"GDP growth {change} by {abs(df_all['GDP growth (%)'].last('Y').values[0])}% year-on-year (yoy) in {df_all['GDP growth (%)'].last('Y').index.year.values[0]}.")

# 4 - Demand-side: Top contribution in the same direction as GDP growth
p.add_run(f" On the demand side, {top_demand_contributions.index[0].lower()} ({top_demand_shares.loc[top_demand_contributions.index[0]].values[0]}% of GDP) contributed the most to {'growth' if df_all['GDP growth (%)'].last('Y').values[0] > 0 else 'contraction'}, with {top_demand_contributions.values[0][0]} percentage points (pp)."
)

# 5 - Supply-side: Top contributions in the same direction as overall growth
p.add_run(f" On the supply side, {top_production_contributions.index[0].lower()} ({top_production_shares.loc[top_production_contributions.index[0]].values[0]}% of GDP) contributed the most to {'growth' if df_all['GDP growth (%)'].last('Y').values[0] > 0 else 'contraction'}, with {top_production_contributions.values[0][0]}pp.")

# Justify paragraph 'p'
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY 

In [7]:
################################################################################################
####                            Make GDP Contribution Graphs                                ####
################################################################################################

# Create the dataframe that will be plotted
df_chart_gdp = df_all[['GDP growth (%)'] + [f'{item} (contribution, pp)' for item in demand_components + production_components]].dropna()
df_chart_gdp.index = df_chart_gdp.index.year
df_chart_gdp.reset_index(inplace=True)

# Create a Pandas Excel writer using xlsxwriter as the engine
writer = pd.ExcelWriter(f'{country}-charts.xlsx', engine='xlsxwriter')
df_chart_gdp[['date', 'GDP growth (%)'] + [f'{item} (contribution, pp)' for item in demand_components]].to_excel(writer, sheet_name='GDP-demand', index=False)
df_chart_gdp[['date', 'GDP growth (%)'] + [f'{item} (contribution, pp)' for item in production_components]].to_excel(writer, sheet_name='GDP-production', index=False)

# Create a blank canvas
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5), sharey=True)
plt.style.use('default')

# Create a stacked bar chart of the components
df_chart_gdp[[f'{item} (contribution, pp)' for item in demand_components]].plot(kind='bar', stacked=True, ax=axs[0])

df_chart_gdp[[f'{item} (contribution, pp)' for item in production_components]].plot(kind='bar', stacked=True, ax=axs[1])

# Create a line plot of the GDP growth series
for i in range(len(axs)):
    axs[i].plot(df_chart_gdp.loc[:, 'GDP growth (%)'], lw=2.5, marker='D', markersize=10, color='black')
    # Set labels in the x-axis
    axs[i].set_xticklabels(df_chart_gdp.date, rotation=0)
    axs[i].set_ylabel('percentage points')
    axs[i].yaxis.set_major_locator(ticker.MaxNLocator(6))
    axs[i].set_xlabel('')
    axs[i].axhline(color='black', linewidth=0.5)

# Set legend
axs[0].legend(['GDP growth (%)'] + demand_components)
axs[1].legend(['GDP growth (%)'] + production_components)

# Set graph formatting and save to local folder
axs[0].set_title('Demand-side contributions to growth')
axs[1].set_title('Supply-side contributions to growth')

plt.tight_layout()
plt.savefig(f'{country}_GDP_components.png')
plt.close()

# Add picture to the document
graph = doc.add_picture(f'{country}_GDP_components.png', width=Cm(14), height=Cm(7))
last_paragraph = doc.paragraphs[-1]
last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

In [8]:
################################################################################################
####                                    Demand-side Section                                 ####
################################################################################################

# 1 - Heading stating largest contributor
change = {}
for item in demand_components:
    if df_all[f'{item} (annual growth, %)'].last('2Y').values[-1] > 0:
        change[f'{item}'] = random.choice(['expanded', 'grew', 'increased', 'picked up', 'jumped'])
    else:
        change[f'{item}'] = random.choice(['decreased', 'contracted', 'shrank', 'declined', 'plunged'])


doc.add_heading(f"{top_demand_growth.index[0]} {change[top_demand_growth.index[0]]} the fastest on the demand side", level=2)

# 2 - Demand-side: Item with the largest growth
p = doc.add_paragraph(f"{top_demand_growth.index[0]} {change[top_demand_growth.index[0]]} by the largest margin ({top_demand_growth.values[0][0]}%).")

# 4 - Other items that grew
i = 1
while np.sign(top_demand_growth.values[i][0]) == np.sign(df_all['GDP growth (%)'].last('Y').values[0]):
    p.add_run(f" {top_demand_growth.index[i]} {change[top_demand_growth.index[i]]} by {top_demand_growth.values[i][0]}%.") 
    i += 1
    if i >= len(top_demand_growth):
        break 

# 5 - Items that contracted
for i in range(1, len(top_demand_growth)):
    if np.sign(top_demand_growth.values[i][0]) != np.sign(df_all['GDP growth (%)'].last('Y').values[0]):
        p.add_run(f" On the other hand, {top_demand_growth.index[i].lower()} {change[top_demand_growth.index[i]]} by {abs(top_demand_growth.values[i][0])}%.")

p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [9]:
################################################################################################
####                                    Supply-side Section                                 ####
################################################################################################

# 1 - Heading stating largest contributor
change = {}
for item in production_components:
    if df_all[f'{item} (annual growth, %)'].last('2Y').values[-1] > 0:
        change[f'{item}'] = random.choice(['expanded', 'grew', 'increased', 'picked up', 'jumped'])
    else:
        change[f'{item}'] = random.choice(['decreased', 'contracted', 'shrank', 'declined', 'plunged'])

doc.add_heading(f"On the supply side, {top_production_growth.index[0].lower()} {change[top_production_growth.index[0]]} the fastest", level=2)

# 2 - Item with the largest growth
p = doc.add_paragraph(f"{top_production_growth.index[0]} {change[top_production_growth.index[0]]} the most rapidly ({top_production_growth.values[0][0]}%).")

# 4 - Other items that grew
i = 1
while np.sign(top_production_growth.values[i][0]) == np.sign(df_all['GDP growth (%)'].last('Y').values[0]):
    p.add_run(f" {top_production_growth.index[i]} {change[top_production_growth.index[i]]} by {top_production_growth.values[i][0]}%.") 
    i += 1
    if i >= len(top_production_growth):
        break 

# 5 - Items that contracted
for i in range(1, len(top_production_growth)):
    if np.sign(top_production_growth.values[i][0]) != np.sign(df_all['GDP growth (%)'].last('Y').values[0]):
        p.add_run(f" On the other hand, {top_production_growth.index[i].lower()} {change[top_production_growth.index[i]]} by {abs(top_production_growth.values[i][0])}%.")

p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [10]:
################################################################################################
####                                Unemployment & Inflation                                ####
################################################################################################

# 1 - Set placeholders & Qualify changes
phillips = ['Unemployment rate (%)', 'Inflation rate (%)']
change = {}
for item in phillips:
    if df_all[item].last('2Y').values[-1] > df_all[item].last('2Y').values[-2]:
        change[item] = random.choice(['increased', 'increased', 'increased'])
    elif df_all[item].last('2Y').values[-1] == df_all[item].last('2Y').values[-2]:
        change[item] = random.choice(['remained', 'stayed'])
    else:
        change[item] = random.choice(['improved', 'declined', 'declined'])

# 2 - Heading stating changes of each
doc.add_heading(f"Unemployment {change['Unemployment rate (%)']}; inflation {change['Inflation rate (%)']}", level=2)

# 3 - Sentence on Unemployment
p = doc.add_paragraph(f"Unemployment {change['Unemployment rate (%)']} from {df_all['Unemployment rate (%)'].last('2Y').values[-2]}% in {df_all['Unemployment rate (%)'].last('2Y').index[-2].year} to {df_all['Unemployment rate (%)'].last('2Y').values[-1]}% in {df_all['Unemployment rate (%)'].last('2Y').index[-1].year},")

# 4 - Sentence on Inflation
p.add_run(f" while inflation {change['Inflation rate (%)']} from {'a deflation of' + str(abs(df_all['Inflation rate (%)'].last('2Y').values[-2])) if df_all['Inflation rate (%)'].last('2Y').values[-2] < 0 else df_all['Inflation rate (%)'].last('2Y').values[-2]}% to {'a deflation of ' + str(abs(df_all['Inflation rate (%)'].last('2Y').values[-1])) if df_all['Inflation rate (%)'].last('2Y').values[-1] < 0 else df_all['Inflation rate (%)'].last('2Y').values[-1]}%.")

if policy_rate_annual.shape[0] > 0:
# 5 - Sentence on CB Policy rate
    p.add_run(f"{' At the end of ' + str(policy_rate_annual.last('Y').index.year[0]) + ', the central bank set the policy rate at ' + str(policy_rate_annual.last('Y').values[0][0]) + '%'}.")

p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [11]:
# ################################################################################################
# ####                                Plot Unemployment & Inflation                           ####
# ################################################################################################

# # Create the dataframe that will be plotted
# df_chart_phillips = pd.DataFrame(df_all[['Unemployment rate (%)', 'Inflation rate (%)']].last('5Y'))
# df_chart_phillips.index = df_chart_phillips.index.year
# df_chart_phillips.reset_index(inplace=True)

# # Create a Pandas Excel writer using xlsxwriter as the engine
# for item in phillips:
#     df_chart_phillips[['date', f'{item}']].to_excel(writer, sheet_name=f'{item}', index=False)

# # Close the Pandas Excel writer and output the Excel file
# writer.save()

# # Create a blank canvas
# fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
# plt.style.use('default')

# # Create a bar chart of inflation and CAB
# df_chart_phillips['Unemployment rate (%)'].plot(kind='bar', ax=axs[0])
# df_chart_phillips['Inflation rate (%)'].plot(kind='bar', ax=axs[1])

# # Set graph formatting
# for i in range(len(axs)):
#     axs[i].set_xticklabels(df_chart_phillips.date, rotation=0)
#     axs[i].set_xlabel('')
#     axs[i].axes.get_yaxis().set_visible(False)

# axs[0].set_title('Unemployment rate (%)')
# axs[1].set_title('Inflation rate (%)')

# # Set data labels for unemployment
# labels_unem = df_chart_phillips['Unemployment rate (%)']
# rects_unem = axs[0].patches

# for rect, label in zip(rects_unem, labels_unem):
#     height = rect.get_height()
#     if height > 0:
#         axs[0].text(rect.get_x() + rect.get_width() / 2, height, str(label) + '%', ha='center', va='bottom')
#     else:
#         axs[0].text(rect.get_x() + rect.get_width() / 2, height - 0.1, str(label) + '%', ha='center', va='bottom')

# # Set data labels for inflation
# labels_inf = df_chart_phillips['Inflation rate (%)']
# rects_inf = axs[1].patches

# for rect, label in zip(rects_inf, labels_inf):
#     height = rect.get_height()
#     if height > 0:
#         axs[1].text(rect.get_x() + rect.get_width() / 2, height, str(label) + '%', ha='center', va='bottom')
#     else:
#         axs[1].text(rect.get_x() + rect.get_width() / 2, height  - 0.2, str(label) + '%', ha='center', va='bottom')

# # Save to local folder
# plt.savefig(f'{country}_unemployment_inflation.png')
# plt.close()

# # Add picture to the document
# graph = doc.add_picture(f'{country}_unemployment_inflation.png', width=Cm(14), height=Cm(7))
# last_paragraph = doc.paragraphs[-1]

# last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

In [12]:
################################################################################################
####                        Current Account Balance Section - Part 1                        ####
################################################################################################

# 1 - Set placeholders
sign_latest = 'surplus' if df_all['Current account balance (% of GDP)'].last('2Y').values[-1] > 0 else 'deficit'
sign_old = 'surplus' if df_all['Current account balance (% of GDP)'].last('2Y').values[-2] > 0 else 'deficit'

# 2 - Heading stating if the CAB is in surplus or deficit
doc.add_heading(f'Current account balance {sign_latest}', level=2)

# 3 - State CAB & Change since last year
p = doc.add_paragraph(f"The current account balance (CAB) recorded a {sign_latest} at {abs(df_all['Current account balance (% of GDP)'].last('2Y').values[-1])}% of GDP in {df_all['Current account balance (% of GDP)'].last('2Y').index[-1].year},")

# 4 - State change since previous year
p.add_run(f" compared to {abs(df_all['Current account balance (% of GDP)'].last('2Y').values[-2])}% in {df_all['Current account balance (% of GDP)'].last('2Y').index[-2].year}.")

p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [13]:
################################################################################################
####                Current Account Balance Section - Download COMTRADE data                ####   ################################################################################################

uitoken = 'b3f9aeb66ba24558248a2c38acebc1f'

# 1 - Download Exports by Partner
url_destination = f"https://comtrade.un.org/api/get/plus?max=100000&type=C&freq=A&px=HS&ps={dt.date.today().year - 2}&r={country_codes.comtrade_code[country]}&p=all&rg=2&cc=TOTAL&uitoken={uitoken}&fmt=csv"
df_destination = pd.read_csv(url_destination)
mask1 = (df_destination['Partner']=='World') & (df_destination['Mode of Transport']=='All MOTs') & (df_destination['2nd Partner']=='World')
total_exports = df_destination[mask1]['Trade Value (US$)'].values[0]
mask2 = (~(df_destination['Partner'] == 'World')) & (df_destination['Mode of Transport'] == 'All MOTs') & (df_destination['2nd Partner']=='World')
df_destination = df_destination[mask2]
df_destination = df_destination.assign(share=lambda x: (x['Trade Value (US$)'] / total_exports * 100).round(1))
top_destination = df_destination.sort_values(by='share', ascending=False)[['Partner', 'share']].reset_index(drop=True).head(5)

# 2 - Download Imports by Partner
url_origin = f"https://comtrade.un.org/api/get/plus?max=100000&type=C&freq=A&px=HS&ps={dt.date.today().year - 2}&r={country_codes.comtrade_code[country]}&p=all&rg=1&cc=TOTAL&uitoken={uitoken}&fmt=csv"
df_origin = pd.read_csv(url_origin)
mask1 = (df_origin['Partner']=='World') & (df_origin['Mode of Transport']=='All MOTs') & (df_origin['2nd Partner']=='World')
total_imports = df_origin[mask1]['Trade Value (US$)'].values[0]
mask2 = (~(df_origin['Partner'] == 'World')) & (df_origin['Mode of Transport'] == 'All MOTs') & (df_origin['2nd Partner']=='World')
df_origin = df_origin[mask2]
df_origin = df_origin.assign(share=lambda x: (x['Trade Value (US$)'] / total_imports * 100).round(1))
top_origin = df_origin.sort_values(by='share', ascending=False)[['Partner', 'share']].reset_index(drop=True).head(5)

# 3 - Download Exports by Product
url_exports = f"https://comtrade.un.org/api/get/plus?max=100000&type=C&freq=A&px=HS&ps={dt.date.today().year - 2}&r={country_codes.comtrade_code[country]}&p=0&rg=2&cc=AG2&uitoken={uitoken}&fmt=csv"
df_exports = pd.read_csv(url_exports)
mask = (df_exports['Mode of Transport']=='All MOTs') & (df_exports['2nd Partner']=='World')
df_exports = df_exports[mask]
df_exports = df_exports.assign(share=lambda x: (x['Trade Value (US$)'] / total_exports * 100).round(1))
top_exports = df_exports.sort_values(by='share', ascending=False)[['Commodity', 'share']].reset_index(drop=True).head(5)

# 4 - Download Imports by Product
url_imports = f"https://comtrade.un.org/api/get/plus?max=100000&type=C&freq=A&px=HS&ps={dt.date.today().year - 2}&r={country_codes.comtrade_code[country]}&p=0&rg=1&cc=AG2&uitoken={uitoken}&fmt=csv"
df_imports = pd.read_csv(url_imports)
mask = (df_imports['Mode of Transport']=='All MOTs') & (df_imports['2nd Partner']=='World')
df_imports = df_imports[mask]
df_imports = df_imports.assign(share=lambda x: (x['Trade Value (US$)'] / total_imports * 100).round(1))
top_imports = df_imports.sort_values(by='share', ascending=False)[['Commodity', 'share']].reset_index(drop=True).head(5)

In [14]:
################################################################################################
####                Current Account Balance Section - Comments on Exports                   ####
################################################################################################

# JULES     > Could you please aggegate EU countries as 1 entity? Same for imports, in the next block
 
# 1 - Top Export Destination
p = doc.add_paragraph(f"{top_destination['Partner'].values[0]} is the top export destination ({top_destination['share'].values[0]}% of exports in {dt.date.today().year - 2}). Other major exports partners include {top_destination['Partner'].values[1]} ({top_destination['share'].values[1]}%), {top_destination['Partner'].values[2]} ({top_destination['share'].values[2]}%), and {top_destination['Partner'].values[3]} ({top_destination['share'].values[3]}%).")

# 3 - Top Export Commodities
p.add_run(f" Top export products are {top_exports['Commodity'].values[0].lower()} ({top_exports['share'].values[0]}% of exports).")

# JULES > Let's keep this part commented out for now as it might be useful later on
# p = doc.add_paragraph(f"Top export commodities include {top_exports['Commodity'].values[0].lower()} ({top_exports['share'].values[0]}% of total exports), {top_exports['Commodity'].values[1].lower()} ({top_exports['share'].values[1]}%), {top_exports['Commodity'].values[2].lower()} ({top_exports['share'].values[2]}%), {top_exports['Commodity'].values[3].lower()} ({top_exports['share'].values[3]}%), and {top_exports['Commodity'].values[4].lower()} ({top_exports['share'].values[4]}%).")

p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [15]:
################################################################################################
####                    Current Account Balance Section - Comments on Imports               ####
################################################################################################

# 1 - Top Import Origins
p = doc.add_paragraph(f"Top imports origins are {top_origin['Partner'].values[0]} ({top_origin['share'].values[0]}%), {top_origin['Partner'].values[1]} ({top_origin['share'].values[1]}%), {top_origin['Partner'].values[2]} ({top_origin['share'].values[2]}%), and {top_origin['Partner'].values[3]} ({top_origin['share'].values[3]}%).")

# 3 - Top Import Commodities
p.add_run(f" Major import commodities are {top_imports['Commodity'].values[0].lower()}({top_imports['share'].values[0]}% of imports).")

# JULES > Let's keep this part commented out for now as it might be useful later on
# p = doc.add_paragraph(f"Major import commodities include {top_imports['Commodity'].values[0].lower()} ({top_imports['share'].values[0]}% of total imports), {top_imports['Commodity'].values[1].lower()} ({top_imports['share'].values[1]}%), {top_imports['Commodity'].values[2].lower()} ({top_imports['share'].values[2]}%), {top_imports['Commodity'].values[3].lower()} ({top_imports['share'].values[3]}%),  and {top_imports['Commodity'].values[4].lower()} ({top_imports['share'].values[4]}%).")

p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [16]:
################################################################################################
####                        Outlook During 2020 (Quarterly Analysis)                        ####
################################################################################################

# 1 - Set Placeholders + Qualify Changes
df_gdp_growth_quarterly = quarterly_data[['GDP growth', 'Household consumption', 'Government expenditures', 'Gross capital formation', 'Exports']].dropna()
change = {}
for item in df_gdp_growth_quarterly.columns:
    if df_gdp_growth_quarterly[item].last('Q').values[0] > 0:
        change[item] = random.choice(['increased', 'picked up', 'rose'])
    elif df_gdp_growth_quarterly[item].last('Q').values[0] == 0:
        change[item] = random.choice(['remained at', 'stayed at'])
    else:
        change[item] = random.choice(['decreased', 'contracted', 'slowed down'])
    
# 2 - Demand-side GDP: Create dataframe for top growth rate items
top_demand_growth_quarterly = []
if df_gdp_growth_quarterly[item].last('Q').values[0] > 0:
    top_demand_growth_quarterly = df_gdp_growth_quarterly.last('Q')[[item for item in df_gdp_growth_quarterly.columns if item != 'GDP growth']].transpose().sort_values(by=df_gdp_growth_quarterly.last('Q').index[0], ascending=False)
else:
    top_demand_growth_quarterly = df_gdp_growth_quarterly.last('Q')[[item for item in df_gdp_growth_quarterly.columns if item != 'GDP growth']].transpose().sort_values(by=df_gdp_growth_quarterly.last('Q').index[0], ascending=True)

# 3 - Heading stating if GDP increased/decreasesd in last quarter w/ available data
p = doc.add_heading(f"Output {'contracted' if df_gdp_growth_quarterly['GDP growth'].last('Q').values[0] < 0 else 'picked up'} in Q{df_gdp_growth_quarterly.last('Q').index.quarter[0]} {df_gdp_growth_quarterly.last('Q').index.year[0]}", level=1)

# 4 - Sentence stating by how much GDP contracted/expanded in last quarter w/ available data
p = doc.add_paragraph(f"Output {'plunged' if df_gdp_growth_quarterly['GDP growth'].last('Q').values[0] < 0 else 'jumped'} by {abs(df_gdp_growth_quarterly['GDP growth'].last('Q').values[0])}% yoy in Q{df_gdp_growth_quarterly.last('Q').index.quarter[0]} {df_gdp_growth_quarterly.last('Q').index.year[0]}")

# 5 - Parenthesis indicating GDP change in the quarter before the last one w/ available data
p.add_run(f" ({df_gdp_growth_quarterly['GDP growth'].last('2Q').values[0]}% in the previous quarter).")

# 6 - Sentence on largest contributor to GDP change in last available quarter
p.add_run(f" {top_demand_growth_quarterly.index[0]} {change[top_demand_growth_quarterly.index[0]]} by the largest margin at {top_demand_growth_quarterly.values[0][0]}% yoy.")

# 7 - Sentence on other large contributors to GDP change in last available quarter
i = 1
while np.sign(top_demand_growth_quarterly.values[i][0]) == np.sign(np.sign(df_gdp_growth_quarterly['GDP growth'].last('Q').values[0])):
    p.add_run(f" {top_demand_growth_quarterly.index[i]} {change[top_demand_growth_quarterly.index[i]]} by {abs(top_demand_growth_quarterly.values[i][0])}%.") 
    i += 1
    if i >= len(top_demand_growth_quarterly) - 2:
        break

p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [17]:
################################################################################################
####                                Consumer Confidence                                     ####
################################################################################################
 
# 1 - Set placeholders & Qualify changes
df_consumer = quarterly_data[['Consumer confidence']].dropna()
if df_consumer.shape != (0, 1):
    
# 2 - Heading  
    p = doc.add_heading(f"Leading indicators", level=2)

# 3 - Paragraph
    if country == 'RUS':
        p = doc.add_paragraph(f"Consumer confidence index was in the {'positive' if df_consumer['Consumer confidence'].last('Q').values[0] > 0 else 'negative'} territory at {df_consumer['Consumer confidence'].last('Q').values[0]} in Q{df_consumer.last('Q').index.quarter[0]} ({df_consumer['Consumer confidence'].last('2Q').values[0]} in Q{df_consumer.last('2Q').index.quarter[0]}).")
    else:
        p = doc.add_paragraph(f"Consumer confidence index was in the {'optimistic' if df_consumer['Consumer confidence'].last('Q').values[0] > 100 else 'pessimistic'} territory at {df_consumer['Consumer confidence'].last('Q').values[0]} in Q{df_consumer.last('Q').index.quarter[0]} ({df_consumer['Consumer confidence'].last('2Q').values[0]} in Q{df_consumer.last('2Q').index.quarter[0]}).")

    p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [18]:
################################################################################################
####                                    Retail Sales                                        ####
################################################################################################

# 1 - Set placeholders & Qualify changes
df_retail = monthly_data[['Retail sales']].dropna()
if (df_retail.shape != (0, 1)) & (df_consumer.shape != (0, 1)):
    
# 2 - Sentence stating the change in Retail sales in the last month w/ available data
    p.add_run(f" Retail sales {'expanded' if df_retail['Retail sales'].last('M').values[0] > 0 else 'shrank'} by {abs(df_retail['Retail sales'].last('M').values[0])}% yoy in {calendar.month_name[df_retail.last('M').index.month[0]]} ({df_retail['Retail sales'].last('2Q').values[0]}% yoy in {calendar.month_name[df_retail.last('2M').index.month[0]]}).")

elif (df_retail.shape != (0, 1)) & (df_consumer.shape == (0, 1)):

    p = doc.add_heading(f"Leading indicators", level=2)

    p = doc.add_paragraph(f"Retail sales {'expanded' if df_retail['Retail sales'].last('M').values[0] > 0 else 'shrank'} by {abs(df_retail['Retail sales'].last('M').values[0])}% yoy in {calendar.month_name[df_retail.last('M').index.month[0]]} ({df_retail['Retail sales'].last('2Q').values[0]}% yoy in {calendar.month_name[df_retail.last('2M').index.month[0]]}).")
    
    p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [19]:
################################################################################################
####                              Industrial production                                     ####
################################################################################################

# 1 - Set placeholders & Qualify changes
df_production = monthly_data[['Industrial production']].dropna()
if (df_production.shape != (0, 1)) & (df_retail.shape != (0, 1)) & (df_consumer.shape != (0, 1)):
    
# 2 - Sentence stating the change in Industrial production in the last month w/ available data
    p.add_run(f" Industrial production {'expanded' if df_production['Industrial production'].last('M').values[0] > 0 else 'shrank'} by {abs(df_production['Industrial production'].last('M').values[0])}% yoy in {calendar.month_name[df_production.last('M').index.month[0]]} ({df_production['Industrial production'].last('2Q').values[0]}% yoy in {calendar.month_name[df_production.last('2M').index.month[0]]}).")

elif (df_production.shape != (0, 1)) & (df_retail.shape == (0, 1)) & (df_consumer.shape == (0, 1)):

    p = doc.add_heading(f"Leading indicators", level=2)

    p= doc.add_paragraph(f"Industrial production {'expanded' if df_production['Industrial production'].last('M').values[0] > 0 else 'shrank'} by {abs(df_production['Industrial production'].last('M').values[0])}% yoy in {calendar.month_name[df_production.last('M').index.month[0]]} ({df_production['Industrial production'].last('2Q').values[0]}% yoy in {calendar.month_name[df_production.last('2M').index.month[0]]}).")

    
    p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [20]:
################################################################################################
####                                   Inflation                                            ####
################################################################################################

# 1 - Set placeholders & Qualify changes
df_inflation = monthly_data[['Inflation']].dropna()
if df_inflation.shape != (0, 1):

# 2 - Heading stating if Inflation increased/decreasesd in last month w/ available data
    p = doc.add_heading(f"Inflation {'picked up' if df_inflation['Inflation'].last('M').values[0] > df_inflation['Inflation'].last('2M').values[0] else 'decreased'}", level=2)

# 3 - Sentence stating the change in Inflation in the last month w/ available data
    p = doc.add_paragraph(f"Inflation {'rose' if df_inflation['Inflation'].last('M').values[0] > df_inflation['Inflation'].last('2M').values[0] else 'stabilized'} to {abs(df_inflation['Inflation'].last('M').values[0])}% yoy in {calendar.month_name[df_inflation.last('M').index.month[0]]} ({df_inflation['Inflation'].last('2M').values[0]}% yoy in {calendar.month_name[df_inflation.last('2M').index.month[0]]}).")
    
    p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [21]:
################################################################################################
####                              Policy rate                                               ####
################################################################################################

# 1 - Set placeholders & Qualify changes
df_policy_rate = monthly_data[['policy_rate_monthly']].dropna()
if df_policy_rate.shape != (0, 1):
    change = ''
    if df_policy_rate['policy_rate_monthly'].last('M').values[0] > df_policy_rate['policy_rate_monthly'].last('2M').values[0]:
        change = random.choice(['increased', 'tightened'])
    elif df_policy_rate['policy_rate_monthly'].last('M').values[0] == df_policy_rate['policy_rate_monthly'].last('2M').values[0]:
        change = random.choice(['maintained', 'kept'])
    else:
        change = random.choice(['decreased', 'eased'])

# 2 - Sentence stating the change in policy_rate_monthly in the last month w/ available data
    p.add_run(f" Meanwhile, the central bank {change} the policy rate to {abs(df_policy_rate['policy_rate_monthly'].last('M').values[0])}% in {calendar.month_name[df_policy_rate.last('M').index.month[0]]}{' ' + '(' + str(df_policy_rate['policy_rate_monthly'].last('2M').values[0]) + '% in ' + str(calendar.month_name[df_policy_rate.last('2M').index.month[0]]) + ')' if df_policy_rate['policy_rate_monthly'].last('M').values[0] != df_policy_rate['policy_rate_monthly'].last('2M').values[0] else ''}.")
    
    p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [22]:

################################################################################################
####                        Consensus Forecast: Growth + Inflation                          ####
################################################################################################

# 1 - Heading
p = doc.add_heading(f"Outlook {'tilted downwards' if country_codes[country_codes.index==country].forecast_gdp_current[0].round(1) < 0 else 'favorable'} this year")

# 2 - Sentence on growth forecast for the current year
p = doc.add_paragraph(f"As of {country_codes[country_codes.index==country].forecast_date[0]}, Consensus Economics panelists project economic growth ending at {country_codes[country_codes.index==country].forecast_gdp_current[0].round(1)}% in {country_codes[country_codes.index==country].forecast_date[0][-4:]}")

# 3 - Sentence on growth forecast for next year
p.add_run(f", while in {int(country_codes[country_codes.index==country].forecast_date[0][-4:]) + 1}, the panelists foresee growth at {country_codes[country_codes.index==country].forecast_gdp_1step[0].round(1)}%.")

# 4 - Sentences on growth drivers forecast and inflation
p.add_run(f" {'Over the same period, consumption is expected to change by ' + str(country_codes[country_codes.index==country].forecast_con_current[0].round(1)) + '% and ' + str(country_codes[country_codes.index==country].forecast_con_1step[0].round(1)) +'%, while investment is projected to change by ' + str(country_codes[country_codes.index==country].forecast_inv_current[0].round(1)) + '% and ' + str(country_codes[country_codes.index==country].forecast_inv_1step[0].round(1)) +'%.' if ~np.isnan(country_codes[country_codes.index==country].forecast_con_current[0].round(1)) else ''} {'Industrial production is seen to change by ' + str(country_codes[country_codes.index==country].forecast_ip_current[0].round(1)) + '% and ' + str(country_codes[country_codes.index==country].forecast_ip_1step[0].round(1)) + '%.' if ~np.isnan(country_codes[country_codes.index==country].forecast_ip_current[0].round(1)) else ''} Finally, Consensus Economics foresee inflation at {country_codes[country_codes.index==country].forecast_inf_current[0].round(1)}% in {country_codes[country_codes.index==country].forecast_date[0][-4:]} and {country_codes[country_codes.index==country].forecast_inf_1step[0].round(1)}% in {int(country_codes[country_codes.index==country].forecast_date[0][-4:]) + 1}.") 

p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [23]:
################################################################################################
####                                    Finalize Document                                   ####
################################################################################################

# Add a line for sources
p = doc.add_paragraph()
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
run = p.add_run(f"Sources: Consensus Economics, The World Bank, UN Comtrade, Haver Analytics, and National Sources. Accessed {dt.date.today().strftime('%d %B %Y')}.")
font = run.font
font.size = Pt(10)
font.italic = True

# Save the .docx file into the local folder
doc.save(f'Country Reports/{country}-Recent-Economic-Developments.docx')